# **Introdução à Ciência de Dados**

**Professor**: Yuri Malheiros

**Grupo**: João Victor Alcoforado de Araújo, Lívia Noêmia Morais Rocha e Matheus Rocha dos Santos Rangel.

## **Projeto #4 - Limpeza e Transformação**

### **Imports e configurações gerais**

In [ ]:
#Dependências
import pandas as pd
import numpy as np

In [ ]:
#Configurações DataFrame
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)
pd.set_option('max_colwidth', None) 

In [ ]:
#Acessando os arquivos do Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##**Limpeza e Transformação - CNVCC** 

[Dicionário de dados do dataset](https://opendatasus.saude.gov.br/dataset/b772ee55-07cd-44d8-958f-b12edd004e0b/resource/38ead83d-b115-4219-852e-7244792bc311/download/dicionario-de-dados-vacinacao.pdf)

In [ ]:
cnvcc = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ICD/Datasets/CNVCC - Nordeste.csv')

In [ ]:
#Formatar data
cnvcc.loc[:, 'vacina_dataAplicacao'] = cnvcc.vacina_dataAplicacao.str[:10]

#Remove erros de cadastro em dataAplicacao
cnvcc = cnvcc[cnvcc['vacina_dataAplicacao'].str[:4] == '2021']

#Ordenação do dataset por data
cnvcc.sort_values('vacina_dataAplicacao', inplace=True)

#Remoção de registros inconsistente, mesmo paciente com mais de um registro de primeira ou segunda dose
#Manteve-se apenas, para cada dose, o registro de data mais antiga
cnvcc.drop_duplicates(subset=['paciente_id', 'vacina_descricao_dose'], keep='first', inplace=True)

#Conversão para datatime
cnvcc['vacina_dataAplicacao'] = pd.to_datetime(cnvcc['vacina_dataAplicacao'], dayfirst=True)

In [ ]:
#Remove instancias sem id
cnvcc = cnvcc[~cnvcc['paciente_id'].isna()]

#Remove outliers de idade
cnvcc = cnvcc[cnvcc['paciente_idade'] < 150]

#Conversão de idade para int
cnvcc = cnvcc.astype({'paciente_idade': np.int32})

In [ ]:
#Remove instancias sem descricao correta da dose
cnvcc = cnvcc[cnvcc['vacina_descricao_dose'] != '\xa0\xa0\xa0\xa0Dose\xa0']

#Simplifica o atributo vacina_descricao_dose
cnvcc.loc[cnvcc['vacina_descricao_dose'] == '\xa0\xa0\xa0\xa01ª\xa0Dose', 'vacina_descricao_dose'] = 1
cnvcc.loc[cnvcc['vacina_descricao_dose'] == '\xa0\xa0\xa0\xa02ª\xa0Dose', 'vacina_descricao_dose'] = 2

In [ ]:
#Padroniza e simplifica o atributo vacina_nome
vacinas = ['Vacina Covid-19 - Covishield', 'Covid-19-Coronavac-Sinovac/Butantan', 'Covid-19-AstraZeneca',
           'Vacina covid-19 - BNT162b2 - BioNTech/Fosun Pharma/Pfizer', 'Vacina covid-19 - Ad26.COV2.S - Janssen-Cilag']

for index, vacina in enumerate(vacinas):
    cnvcc.loc[cnvcc['vacina_nome'] == vacina, 'vacina_nome'] = index+1

In [ ]:
#Denormalização do dataset
#Organização sobre primeira e segunda dose em um único registro

#Dataset apenas com os dados sobre o paciente
dataset_denormalizado = cnvcc[['paciente_id', 'paciente_idade', 'paciente_enumSexoBiologico',
                              'estabelecimento_uf', 'vacina_categoria_nome']]

#Remoção de duplicados, para ter apenas um registro por paciente
dataset_denormalizado = dataset_denormalizado.drop_duplicates(subset='paciente_id')

primeira_dose_colunas = {'vacina_dataAplicacao': 'data_aplicacao_1', 'vacina_nome': 'nome_vacina_1'}
segunda_dose_colunas = {'vacina_dataAplicacao': 'data_aplicacao_2', 'vacina_nome': 'nome_vacina_2'}

#Dataset primeira dose
primeira_dose = cnvcc[cnvcc['vacina_descricao_dose'] == 1]
primeira_dose = primeira_dose[['paciente_id', 'vacina_dataAplicacao', 'vacina_nome']]
primeira_dose = primeira_dose.rename(columns=primeira_dose_colunas)
primeira_dose['primeira_dose'] = 1

#Dataset segunda dose
segunda_dose = cnvcc[cnvcc['vacina_descricao_dose'] == 2]
segunda_dose = segunda_dose[['paciente_id', 'vacina_dataAplicacao', 'vacina_nome']]
segunda_dose = segunda_dose.rename(columns=segunda_dose_colunas)
segunda_dose['segunda_dose'] = 1

#Outer join dos datasets, para formar um dataset com informações sobre o paciente, primeira dose e segunda dose
merged = pd.merge(dataset_denormalizado, primeira_dose, on='paciente_id', how='outer')
cnvcc = pd.merge(merged, segunda_dose, on='paciente_id', how='outer')
 
#Preenche os nulos para indicar que a dose não foi tomada
cnvcc = cnvcc.fillna({'primeira_dose': 0, 'nome_vacina_1': 0, 'segunda_dose': 0, 'nome_vacina_2': 0})

#Desaloca variáveis
del primeira_dose
del segunda_dose
del dataset_denormalizado

In [ ]:
#Renomeação das colunas
novas_colunas = {'paciente_id':'id', 'paciente_idade':'idade', 'paciente_enumSexoBiologico': 'sexo', 
                 'estabelecimento_uf': 'estado', 'vacina_categoria_nome': 'categoria_vacinacao'}

cnvcc = cnvcc.rename(columns=novas_colunas)

#Ordenação das colunas
cnvcc = cnvcc[['id', 'idade', 'sexo', 'estado', 'categoria_vacinacao', 
               'primeira_dose', 'data_aplicacao_1', 'nome_vacina_1',
               'segunda_dose', 'data_aplicacao_2', 'nome_vacina_2']]

In [ ]:
#Define os tipos de dados das colunas
data_types = {'idade':np.int32, 'primeira_dose':np.int32, 'nome_vacina_1':np.int32, 'segunda_dose':np.int32, 'nome_vacina_2':np.int32}
cnvcc = cnvcc.astype(data_types)

In [ ]:
#Preenche os nulos da coluna "categoria_vacinacao"
cnvcc.loc[cnvcc['categoria_vacinacao'].isnull(), 'categoria_vacinacao'] = 'Não Informada'

In [ ]:
cnvcc.head(30)

,id,idade,sexo,estado,categoria_vacinacao,primeira_dose,data_aplicacao_1,nome_vacina_1,segunda_dose,data_aplicacao_2,nome_vacina_2
0,80a9a6b3b420e62ef72af97109c2b5e19c31be5a51443e98ade139d5c0b4bba7,40,M,BA,Trabalhadores de Saúde,1,2021-01-01,2,1,2021-03-03,2
1,ade18a9f602f43b0913e58dd0017ab948ade12b9d93dfadbb1c671ef08c6184d,59,F,RN,Trabalhadores de Saúde,1,2021-01-01,1,1,2021-04-12,1
2,b9df4cc60d2a4ca7322487617364dbf488507ab76f5e75693c4ec19e608dbff3,28,M,CE,Trabalhadores de Saúde,1,2021-01-01,2,1,2021-02-22,2
3,540ed555c1d833a469d4ab4fc50e8d521eb13987d52f0bb146786913d4842714,42,F,BA,Trabalhadores de Saúde,1,2021-01-01,2,1,2021-03-01,2
4,7149bde02e21ff14d09cebe2e811b8b825754e8ed3fbe78c78efd0966a6dd91c,46,F,BA,Trabalhadores de Saúde,1,2021-01-01,2,1,2021-03-01,2
5,5c0752950b1e20250950a84478ff5a24b98d36915cfb5e21d9ab831917ce6879,46,M,RN,Trabalhadores de Saúde,1,2021-01-01,1,1,2021-04-12,1
6,fe7dd22328a52ce3ef71274e0cf836fc4ddb51507b34e2f2574bca21e1401ed7,48,F,BA,Trabalhadores de Saúde,1,2021-01-01,2,1,2021-03-01,2
7,7254326c7cad3702ec29b3ceca10a7c5fcb2308c04a07f03c0bbcef17c37f226,68,F,RN,Faixa Etária,1,2021-01-02,2,1,2021-04-18,2
8,7b468fae97be986f72ad92e64cc35534ce4291c52dcd6906459fb915b5e5304e,59,M,RN,Comorbidades,1,2021-01-02,1,0,NaT,0
9,1cbdba8b28b7d1b738048a4341c41652097dafdbe11df07a84492e53a3d69887,65,F,MA,Faixa Etária,1,2021-01-02,2,1,2021-02-22,2


In [ ]:
cnvcc.to_csv('Datasets/CNVCC - Nordeste2.csv', index=False)